In [13]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

from scipy.special import spherical_jn as j_l
from scipy.special import spherical_in as i_l
from spherical_bessel_zeros import Jn_zeros
from scipy.integrate import quadrature

from dataset_processing import get_dataset_slice
from LE_ps import get_LE_expansion, get_LE_ps

In [14]:
a = 6.0  # Radius of the sphere
E_max_2 = 300

In [15]:
l_big = 26
n_big = 26

z_ln = Jn_zeros(l_big, n_big)  # Spherical Bessel zeros
z_nl = z_ln.T

E_nl = z_nl**2
E_max = E_max_2 - E_nl[0, 0]
n_max = np.where(E_nl[:, 0] <= E_max)[0][-1] + 1
l_max = np.where(E_nl[0, :] <= E_max)[0][-1]
print(n_max, l_max)

5 11


In [16]:
def R_nl(n, l, r):
    return j_l(l, z_nl[n, l]*r/a)

def N_nl(n, l):
    # Normalization factor for LE basis functions
    def function_to_integrate_to_get_normalization_factor(x):
        return j_l(l, x)**2 * x**2
    integral, _ = sp.integrate.quadrature(function_to_integrate_to_get_normalization_factor, 0.0, z_nl[n, l])
    return (1.0/z_nl[n, l]**3 * integral)**(-0.5)

def get_LE_function_python(n, l, r):
    R = np.zeros_like(r)
    for i in range(r.shape[0]):
        R[i] = R_nl(n, l, r[i])
    return N_nl(n, l)*R*a**(-1.5)

In [17]:
# Feed LE (delta) radial spline points to Rust calculator:

n_spline_points = 101
spline_x = np.linspace(0.0, a, n_spline_points)  # x values

spline_f = []
for l in range(l_max+1):
    for n in range(n_max):
        spline_f_single = get_LE_function_python(n, l, spline_x)
        spline_f.append(spline_f_single)
spline_f = np.array(spline_f).T
spline_f = spline_f.reshape(n_spline_points, l_max+1, n_max)  # f(x) values

In [18]:
def get_LE_function_derivative(n, l, r):
    delta = 1e-6
    all_derivatives_except_at_zero = (get_LE_function_python(n, l, r[1:]+delta) - get_LE_function_python(n, l, r[1:]-delta)) / (2.0*delta)
    derivative_at_zero = (get_LE_function_python(n, l, np.array([delta/10.0])) - get_LE_function_python(n, l, np.array([0.0]))) / (delta/10.0)
    return np.concatenate([derivative_at_zero, all_derivatives_except_at_zero])

spline_df = []
for l in range(l_max+1):
    for n in range(n_max):
        spline_df_single = get_LE_function_derivative(n, l, spline_x)
        spline_df.append(spline_df_single)
spline_df = np.array(spline_df).T
spline_df = spline_df.reshape(n_spline_points, l_max+1, n_max)  # df/dx values

In [19]:
with open("splines.txt", "w") as file:
    np.savetxt(file, spline_x.flatten(), newline=" ")
    file.write("\n")

with open("splines.txt", "a") as file:
    np.savetxt(file, (1.0/(4.0*np.pi))*spline_f.flatten(), newline=" ")
    file.write("\n")
    np.savetxt(file, (1.0/(4.0*np.pi))*spline_df.flatten(), newline=" ")
    file.write("\n")

In [21]:
train_slice = "0:100"
train_structures = get_dataset_slice("random-ch4-10k.extxyz", train_slice)

ps = get_LE_ps(train_structures, "splines.txt", E_nl, E_max_2, a)
print(ps)

Reading dataset
Shuffling and extracting from dataset (length: 10000)
Shuffling and extraction done
TensorMap with 1 blocks
keys: ['_']
        0
